In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import os
import json
import tempfile
import requests
import numpy as np

import tensorflow as tf

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.15.0


# **Add TensorFlow Serving Distribution URI as a Package Source**

Kita akan menggunakan TensorFlow Serving menggunakana `Aptitude` (the default Debian package manager) Karena Colab runs di Debian environment.

Sebelum kita dapat menginstal Tensorflow Serving, kita perlu menambahkan paket `tensorflow-model-server` ke dalam daftar paket yang diketahui Aptitude. Perhatikan bahwa kita running sebagai root.

**Note**: Notebook ini menjalankan tensorflow serving secara native, tetapi [Anda juga dapat menjalankannya dalam Docker Container](https://www.tensorflow.org/tfx/serving/docker?hl=id), yang merupakan salah satu cara termudah mulai menggunakan tensorflow serving. Docker engine tersedia untuk berbagai platform `Linux`,`Windows`, dan `Mac`.

In [3]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0  17201      0 --:--:-- --:--:-- --:--:-- 17311
OK
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,026 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.dow

# **Install TensorFlow Serving**

Sekarang setelah paket Aptitude telah diperbarui, kita dapat menggunakan perintah `apt-get` untuk menginstal Tensorflow model server.

In [4]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 650 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.16.1 [650 MB]
Fetched 650 MB in 6s (101 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.16.1_all.deb ...
Unpacking tensorflow-model-server (2.16.1) ...
Setting up tensorflow-model-server (2.16.1) ...


# **Create Dataset**

Sekarang, kita akan membuat sebuah dataset sederhana yang mengekspresikan hubungan tersebut:

*y* = 2x - 1

antara inputs (`xs`) dan outputs (`ys`).

In [5]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# **Build dan Train the Model**

Kita akan menggunakan model yang paling sederhana untuk contoh ini. Karena kita akan train model kita untuk `500 epochs`, untuk menghindari clutter/kekacauan pada layar, kita akan menggunakan argumen `verbose = 0` dalam metode `fit.` Mode Verbosity bisa:


*   `0`: Silent.
*   `1`: Progress bar.
*   `2`: one line per epoch.

Sebagai catatan tambahan, kami harus menyebutkan bahwa karena progress bar tidak terlalu berguna ketika masuk ke sebuah file, maka `verbose = 2` direkomendasikan ketika tidak berjalan secara interaktif (misalnya, dalam production environment).



In [6]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])

model.compile(optimizer='sgd',
              loss='mean_squared_error')

history = model.fit(xs, ys, epochs=500, verbose=0)

print("Finished training the model")

Finished training the model


# **Test the Model**

Setelah model di train, kita dapat mengetestnya. Jika kita memberi nilai `10`, kita akan mendapatkan nilai yang sangat mendekati `19`.

In [7]:
print(model.predict([10.0]))

1/1 [==============================] - 0s 128ms/step
[[18.981976]]


# **Save the Model**

untuk meload model yang telah dilatih ke dalam tensorflow serving, pertama-tama kita harus menyimpannya ke dalam format `SavedModel`. Ini akan membuat file protobuf dalam hirarki direktori yang terdefinisi dengan baik, dan akan menyertakan `nomor versi`.

Tensorflow Serving memungkinkan kita memilih versi model yang mana, atau "Servable" yang ingin kita gunakan ketika membuat inference requests/permintaan inferensi. Setiap versi akan di ekspor ke Sub-direktori yang berbeda di bawah jalur/path yang diberikan.

In [8]:
MODEL_DIR = tempfile.gettempdir()

version = 1

export_path = os.path.join(MODEL_DIR, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

model.save(export_path, save_format="tf")

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}


export_path = /tmp/1
total 60
drwxr-xr-x 2 root root  4096 May 17 14:17 assets
-rw-r--r-- 1 root root    55 May 17 14:17 fingerprint.pb
-rw-r--r-- 1 root root  4422 May 17 14:17 keras_metadata.pb
-rw-r--r-- 1 root root 39579 May 17 14:17 saved_model.pb
drwxr-xr-x 2 root root  4096 May 17 14:17 variables


# **Examine Your Saved Model**

Kita akan menggunakan command line utility `saved_model_cli` untuk melihat `MetaGraphDefs` dan `SignatureDefs` di SavedModel kita. Definisi signature didefinisikan oleh tensor input dan output, dan disimpan dengan default serving key.

In [9]:
!saved_model_cli show --dir {export_path} --all

2024-05-17 14:21:36.125951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 14:21:36.126033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 14:21:36.132164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 14:21:38.398960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the followin

# **Run the TensorFlow Model Server**

Sekarang kita akan lunch/meluncurkan Tensorflow model server dengan bash script. Kita akan menggunakan argumen `--bg` untuk menjalankan script di latar belakang.

Script kita akan mulai memrunning TensorFlow Serving dan memload model kita. Berikut adalah parameter yang akan digunakan:



*   `rest_api_port`: Port yang akan anda gunakan untuk requests/permintaan
*   `model_name`: Anda akan menggunakan ini dalam URL requests/permintaan anda. dan bisa apa saja.
*   `model_base_path`: Ini adalah jalur/path ke direktori tempat anda menyimpan mode anda.

Selain itu, karena variabel yang menunjuk ke direktori yang berisi model ada di Python, kita perlu cara untuk memberi tahu Bash script di mana menemukan model. Untuk melakukan ini, kita akan menulis nilai variabel Python ke environment variable menggunakan fungsi `os.environ`.

In [10]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [11]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=helloworld \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Sekarang kita dapat melihat `Log server`

In [12]:
!tail server.log

[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 250] NET_LOG: Entering the event loop ...


# **Create JSON Object with Test Data**

Kita sekarang siap untuk membangun JSON object dengan beberapa data sehingga kita dapat membuat beberapa inferences/kesimpulan. Kita akan menggunakan dan sebagai test data kita.

In [13]:
xs = np.array([[9.0], [10.0]])
data = json.dumps({"signature_name": "serving_default", "instances": xs.tolist()})
print(data)

{"signature_name": "serving_default", "instances": [[9.0], [10.0]]}


# **Make Inference Request**

Terakhir, kita dapat membuat requests/permintaan inference dan mendapatkan inferencenya kembali. Kita akan mengirimkan permintaan prediksi sebagai POST ke titik akhir REST server kita, dan meneruskannya dengan test data kita. Kita akan meminta server kita untuk memberikan versi terbaru dari model kita tanpa menentukan versi tertentu. Responnya akan berupa JSON yang berisi prediksi.

In [14]:
# if this cell fails execution because of an "...Failed to establish a new connection..." error,
# try replacing in the link below 'localhost' with '127.0.0.1'

headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/helloworld:predict', data=data, headers=headers)

print(json_response.text)

{
    "predictions": [[16.9845886], [18.9819756]
    ]
}


Kita juga dapat melihat prediksi secara langsung dengan memuat nilai untuk kunci prediksi.

In [15]:
predictions = json.loads(json_response.text)['predictions']
print(predictions)

[[16.9845886], [18.9819756]]
